In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.utils import to_categorical
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Load data from CSV
data = pd.read_csv('/content/drive/MyDrive/amazon_data_refined1.csv', error_bad_lines=False, low_memory=False)

# Data preprocessing
stemmer = SnowballStemmer("english")
stopword = set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

data['Combined_Text'] = data['Reviews'].astype(str)
text_data = data['Combined_Text'].apply(lambda x: str(x)).apply(clean)

Mounted at /content/drive


<ipython-input-3-0ec6ac3781cd>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/drive/MyDrive/amazon_data_refined1.csv', error_bad_lines=False, low_memory=False)


In [4]:
# Convert sentiment labels to numerical format
label_mapping = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
data['Output'] = data['Output'].map(label_mapping)
labels = data['Output'].values
print(labels)
# Tokenize text data
max_words = 10000
max_seq_length = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
X = pad_sequences(sequences, maxlen=max_seq_length)

# Convert sentiment labels to one-hot encoding
num_classes = 3
y = to_categorical(labels, num_classes=num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model architecture
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_seq_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

[2 2 2 ... 2 2 2]
Epoch 1/5
230/230 [==============================] - 49s 164ms/step - loss: 0.5300 - accuracy: 0.7932 - val_loss: 0.3269 - val_accuracy: 0.8857
Epoch 2/5
230/230 [==============================] - 16s 69ms/step - loss: 0.2229 - accuracy: 0.9287 - val_loss: 0.2723 - val_accuracy: 0.9154
Epoch 3/5
230/230 [==============================] - 8s 33ms/step - loss: 0.1261 - accuracy: 0.9627 - val_loss: 0.2535 - val_accuracy: 0.9246
Epoch 4/5
230/230 [==============================] - 8s 33ms/step - loss: 0.0742 - accuracy: 0.9814 - val_loss: 0.2657 - val_accuracy: 0.9295
Epoch 5/5
115/115 [==============================] - 0s 3ms/step - loss: 0.2792 - accuracy: 0.9287
Test Loss: 0.27921703457832336
Test Accuracy: 0.928726851940155
